# Distributed Systems
## 2021/22

Lab 8

Nuno Preguiça, Sérgio Duarte, Dina Borrego, João Vilalonga

# Goals

In the end of this lab you should be able to:

+ Understand what is OAuth
+ How to register an application with Dropbox
+ How to generate the credentials needed for OAuth in Dropbox 
+ How to take advantage of the REST API documentation of Dropbox
+ Know how to make requests to Dropbox using OAuth using the library ScribeJava

# OAuth

Many online services only allow access through secure channels with client authentication.

+ Secure channels are provided by SSL/TLS;
+ Client authentication is provided by OAuth.

OAuth allows users to provide access to resources stored in an external service (eg., Dropbox) to
be accessed securely by some other application, without sharing the users'
credentials.

# OAUTH IN THE CONTEXT OF DISTRIBUTED SYSTEMS

---

## OAUTH Application registration

Applications that wish to use user’s resources in some external service must register with that service.

For example, a web application designed to access the user's own Dropbox will have to be
a registered application at Dropbox.

• This step creates the authentication pieces for that specific application: **API KEY** and **API SECRET**<br>
     -- required arguments for the Dropbox API.

## OAUTH User Authorization

To autorize an web application, the user is required to authenticate with the external service and only then enable access to its resources to that particular application.

For example, if the external service is Dropbox...

+ The user is directed to a login page at Dropbox; 
+ The user signs in to her account, using her usual Dropbox account credentials; 
+ If the login is sucessful, the user is presented with a dialog to review and agree to the type of access being asked by the web application to her Dropbox account;

+ This authorization process returns an **ACCESS TOKEN** for the (user, web application) pair.
-- this token included in the Dropbox API requests, authenticates the web application.

The user's Dropbox account credentials are **not shared** with the web application!

### Note

In this course, we will make it simpler by combining the two prior steps. 

Basically, we will generate the **API KEY**, **API SECRET** and **ACCESS TOKEN** 
offline.

This is often possible when the **developer** of the application
accessing the external service via OAUTH is **authorizing access
to her own account**.

## Register an Application with Dropbox

1. If you do not have one, create a free Dropbox account [here](https://www.dropbox.com/basic)
2. Create a Dropbox App [here](https://www.dropbox.com/developers)
 1. Select: **Scoped access**
 2. Select: **App Folder** <br>
     -- good to avoid messing up your regular account...
 3. Name your app <br> 
     -- eg., sd2122_XXXXX_YYYYY
 4. Click on **Create app**
 
3. Retrieve your **App Key**, **App secret** and generate the **Access Token**<br>
    -- default values are fine.
4. In the **Permissions** tab, enable all permissions in **Files and Folders**
5. Press **Submit** and you are done.

Your Dropbox apps can be managed [here](https://www.dropbox.com/developers/apps?_tk=pilot_lp&_ad=topbar4&_camp=myapps). Use this console to delete an app, or
to find its OAuth details again.


# Dropbox API Documentation

Dropbox API is very well [documented](https://www.dropbox.com/developers/documentation/http/overview).

The API is REST-like but does not follow exactly REST conventions,
including those adopted in this course.

Some important observations:

+ Most operations in the API are POST operations, using the body of the HTTP request to pass arguments to operations;

+ Requests often require HTTP headers, in some cases also to pass arguments;

+ Successful requests return OK on success; they do not return NO_CONTENT on *void*. 

+ Analyze and follow the documentation carefully to avoid receiving 400 BAD REQUEST replies from the service.<br>
    -- When this happens, the reply body usually provides an explanation of why the operation failed.

## Dropbox API Referece

<img src="https://preguica.github.io/sd2122/praticas2122/aula8/api_create_folder.png"><img>


Note: Read the API documentation carefully! 
The Dropbox API is very strict and will return BAD REQUEST to malformed requests. 

## Dropbox - Create Folder Endpoint

To create a new Dropbox folder, we use the [/create_folder](https://www.dropbox.com/developers/documentation/http/documentation#files-create_folder) endpoint.

Key pieces of information, as detailed in the documentation:

The endpoint URL: [https://api.dropboxapi.com/2/files/create_folder_v2]()

The parameters and the result, are both encoded as **JSON**, <br> matching
the required **Content-type: application/json** request header,
as part of the HTTP request.

### Parameters

```json
{
    "autorename": false,
    "path": "/Homework/math"
}
```


### Returns

```json
{
    "metadata": {
        "id": "id:a4ayc_80_OEAAAAAAAAAXz",
        "name": "math",
        "path_display": "/Homework/math",
        "path_lower": "/homework/math",
        "property_groups": [
            {
                "fields": [
                    {
                        "name": "Security Policy",
                        "value": "Confidential"
                    }
                ],
                "template_id": "ptid:1a5n2i6d3OYEAAAAAAAAAYa"
            }
        ],
        "sharing_info": {
            "no_access": false,
            "parent_shared_folder_id": "84528192421",
            "read_only": false,
            "traverse_only": false
        }
    }
}
```

## OAUTH Requests to Dropbox in JAVA

In Java, OAuth requests are straightforward using a third party library, such as [ScribeJava](https://github.com/scribejava/scribejava)

ScribeJava requires additional Maven dependencies in the project's **pom.xml**
```xml
<dependency>
    <groupId>com.google.code.gson</groupId>
    <artifactId>gson</artifactId>
    <version>2.9.0</version>
</dependency>
<dependency>
    <groupId>com.github.scribejava</groupId>
    <artifactId>scribejava-apis</artifactId>
    <version>8.3.1</version>
</dependency>
<dependency>
    <groupId>org.pac4j</groupId>
    <artifactId>pac4j-oauth</artifactId>
    <version>5.4.3</version>
</dependency>
```

Gson is JSON encoder/decoder library from Google. Will
be used to encode operation arguments and decode the results.

The other two dependencies are for ScribeJava.

## ScribeJava: Initialization...

```java
private static final String apiKey = "INSERT YOURS";
private static final String apiSecret = "INSERT YOURS";
private static final String accessTokenStr = "INSERT YOURS";

var json = new Gson();
var accessToken = new OAuth2AccessToken(accessTokenStr);
var service = new ServiceBuilder(apiKey).apiSecret(apiSecret).build(DropboxApi20.INSTANCE);
```

## Dropbox CreateFolder arguments

We simply represent arguments as Java records that we later encode into JSON.

```java
public record CreateFolderV2Args(String path, boolean autorename) {
}
```

## ScribeJava: Prepare Request 

```java
private static final String CONTENT_TYPE_HDR = "Content-Type";
private static final String JSON_CONTENT_TYPE = "application/json; charset=utf-8";


var createFolder = new OAuthRequest(Verb.POST, CREATE_FOLDER_V2_URL);
createFolder.addHeader(CONTENT_TYPE_HDR, JSON_CONTENT_TYPE);

var json_args = json.toJson(new CreateFolderV2Args(directoryName, false);
createFolder.setPayload( json_args ));

service.signRequest(accessToken, createFolder);
```

The *signRequest* call appends the HTTP header: `Authorization: Bearer ...` to the request
described in Dropbox API. Without it the operation will fail.

## ScribeJava: Issue Request 


```java
private static final int HTTP_SUCCESS = 200;

var response = service.execute(createFolder);
if (response.getCode() != HTTP_SUCCESS) 
    throw new RuntimeException(String.format("Failed to create directory: %s, Status: %d, \nReason: %s\n",
                                             directoryName, response.getCode(), response.getBody()));
```

The response let us know what happened via the HTTP status code.<br>
If the operation failed, the HTTP body explains why. 

## ScribeJava: Decoding the result

A quick way to get the result is to decode it from JSON into some kind of Java Map.
```java
var result = json.fromJson( r.getBody(), HashMap.class);
System.out.println( result);
```

Often it more convenient to decode the result into some specific Java
class that models the response exactly. 

```java
var result = json.fromJson( r.getBody(), SomeClass.class);
System.out.println( result);
```

Check the [sample code](https://preguica.github.io/sd2122/praticas2122/aula8/sd2122-aula8.zip) provided for this class
for an example, using the [ListFolder](https://www.dropbox.com/developers/documentation/http/documentation#files-list_folder) endpoint.

In some operations the result is not encoded as JSON, and we need the raw binary response, <br>for instance when downloading a file. In such cases, the result is available from the response as a *stream*.

```java
    var in = r.getStream();
```

# EXERCISES

1. Try the sample code, after you created your Dropbox app details: api, secret and access token, as described above.
2. Study the Dropbox API on how to create/update, read and delete files/folders.
3. Create a new version of your project's Files service that stores its files in a Dropbox account.
4. Try your new version of the Files service using the Tester, just do not forget to update .props file
to use the new class.